In [1]:
# installs go here
!pip install sklearn
!pip install sklearn-crfsuite
!pip install pandas

  Using cached sklearn-0.0.tar.gz (1.1 kB)
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


You should consider upgrading via the 'C:\Users\smkea\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\smkea\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\smkea\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [1]:
""" Function for handling referring expressions""" # what does breakpoint mean? Hover over red dot
# put all imports here to make it simple?
# had to install it like this instead of import
import pandas
import os
# intent classifier imports:
from pandas import DataFrame

import numpy
from pandas import DataFrame
# FIX THE TRAINING PART!
#from sklearn.feature_extraction.text import CountVectorizer # has a problem finding this, something to do w enabling long paths on my machine????
#from sklearn.linear_model import SGDClassifier
#from sklearn.pipeline import Pipeline
import random
import re
import sys # need this one?
#import sklearn, sklearn_crfsuite
import sklearn_crfsuite

referring_questions = ["What size pizza would you like? We have small, medium, and large.\n",
                       "Do you want your pizza to be small, medium, or large?\n",
                       "What type of crust would you like? We have regular, thin, deep dish, and gluten free.\n", 
                       "Would you like regular, thin, deep dish, or gluten free crust?\n",
                       "Would you like pickup or delivery?\n", "Pickup or delivery?\n",
                       "Is this order for pickup or delivery?\n"]

def referring_expressions(utterance, sf, crf):
  correct = 0
  lsu = sf.last_system_utterance
  if lsu in referring_questions[0:2]:
    #print("Here")
    for scm in size_choices_mapping:
      if scm in utterance:
        answer = input(size_choices_mapping[scm].capitalize() + "?\n")
        correct = 0
        for item in confirm:
          if item in answer:
            print("Okay, great! ", end = "") #don't start new line
            sf.Slots['pizzas'][sf.pizza_num].info['pizza_size'] = size_choices_mapping[scm]
            correct = 1
            break
        if correct == 0:
            print("Sorry. ", end = '')
        break

  elif lsu in referring_questions[2:4]:
    for ccm in crust_choices_mapping:
      if ccm in utterance:
        answer = input(crust_choices_mapping[ccm].capitalize() + "?\n")
        correct = 0
        for item in confirm:
          if item in answer:
            print("Okay, great! ", end = "") #don't start new line
            sf.Slots['pizzas'][sf.pizza_num].info['crust_type'] = crust_choices_mapping[ccm]
            correct = 1
            break
        if correct == 0:
            print("Sorry. ", end = '')
        break

  elif lsu in referring_questions[4:]:
    for dcm in delivery_choices_mapping:
      if dcm in utterance:
        answer = input(delivery_choices_mapping[dcm].capitalize() + "?\n")
        correct = 0
        for item in confirm:
          if item in answer:
            print("Okay, great! ", end = "") #don't start new line
            sf.Slots['delivery_option'] = delivery_choices_mapping[dcm]
            correct = 1
            break
        if correct == 0:
            print("Sorry. ", end = '')
        break

  return correct

#system_knowledge.py **********************************************************************************************************************
# put this block here instead of importing system_knowledge as a library
"""
Information the system has to know - menu items and prices, questions it has to ask, possible wrong/misspelled inputs from users, etc. 
"""

pizza_types = ['hawaiian', 'meat lovers', '4 cheese', 'four cheese', 'pepperoni', 'veggie supreme', 'vegan']
pizza_crusts = ['thin', 'regular', 'deep dish', 'gluten free', 'stuffed']
pizza_si4zes = ['small', 'medium', 'large']
delivery_options = ['pickup', 'delivery']
confirm = ['yes', 'yeah', 'yep', 'okay', 'kay', 'ok']
deny = ['no', 'nope', 'no way']
hello = ['hi', 'hello', 'hey']
goodbye = ['bye', 'goodbye']

#Knowledge graphs - change these pizzas, they make no sense
SpecialtyPizzas = {'hawaiian': ['pineapple', 'ham', 'mozzarella'],
                   'meat lovers': ['mozzarella', 'pepperoni', 'ham', 'bacon', 'sausage'],
                   '4 cheese': ['mozzarella', 'cheddar', 'swiss', 'provolone'],
                   'four cheese': ['mozzarella', 'cheddar', 'swiss', 'provolone'],
                   'pepperoni': ['mozzarella', 'pepperoni'],
                   'veggie supreme': ['mozzarella', 'green peppers', 'red onions', 'mushrooms', 'black olives'],
                   'vegan': ['green peppers', 'red onions', 'mushrooms', 'black olives']
                   }

Toppings = {'mozzarella': 1.50, 'cheddar': 1.50, 'swiss': 1.50, 'provolone': 1.50, 'pineapple': 2.00,
            'green peppers': 2.00, 'red onions': 2.00, 'mushrooms': 2.00, 'black olives': 2.00,
            'pepperoni': 2.50, 'ham': 2.50, 'bacon': 3.00, 'sausage': 3.00}

# repetitive?
#PizzaCrusts = {'small': {'thin': 10.00, 'regular': 10.00, 'deep dish': 12.00, 'gluten free': 15.00},
               #'medium': {'thin': 12.00, 'regular': 12.00, 'deep dish': 14.00, 'gluten free': 18.00},
               #'large': {'thin': 14.00, 'regular': 14.00, 'deep dish': 16.00, 'gluten free': 21.00}
               #}

PizzaSizes = {'small': '10"', 'medium': '12"', 'large':'14"'}

Sides = {'garlic knots': 7.00, 'mozzarella sticks': 9.00, 'green salad': 10.00, 'caesar salad': 12.00}

Drinks = {'cola': 3.50, 'root beer': 3.50, 'orange soda': 3.50, 'lemon soda': 3.50,
          'mineral water': 4.50, 'ginger ale': 3.50}
# why don't we need this?
#DeliveryOptions = {'pickup': 0.00, 'delivery': 3.00}

# dictionary to store crust and size-based prices
PizzaCrusts = {'small': {'thin': 10.00, 'regular': 10.00, 'deep dish': 12.00, 'stuffed': 2.00, 'gluten free': 15.00},
               'medium': {'thin': 12.00, 'regular': 12.00, 'deep dish': 14.00, 'stuffed': 2.00,'gluten free': 18.00},
               'large': {'thin': 14.00, 'regular': 14.00, 'deep dish': 16.00, 'stuffed': 2.00,'gluten free': 21.00},
               '10"': {'thin': 10.00, 'regular': 10.00, 'deep dish': 12.00, 'stuffed': 2.00,'gluten free': 15.00},
               '12"': {'thin': 12.00, 'regular': 12.00, 'deep dish': 14.00, 'stuffed': 2.00,'gluten free': 18.00},
               '14"': {'thin': 14.00, 'regular': 14.00, 'deep dish': 16.00, 'stuffed': 2.00,'gluten free': 21.00}
               }

# toy dictionary for regular customers who order a usual. Phone number: Name: address
regulars = {'6311231234': {'pizza_type': 'pepperoni', 'pizza_size': 'large', 'crust_type': 'thin',
              'delivery_option': '', 'phone_number': '6311231234', 'address': '123 Elm St.', 'name':'Tony'},
            '4871231234': {'pizza_type': 'veggie supreme', 'pizza_size': 'small', 'crust_type': 'gluten free',
              'delivery_option': '', 'phone_number': '4871231234', 'address': '123 Main St.', 'name':'Rebecca'}}

# toy dictionary for querying order status
current_orders = {'6311231234': {'name': 'Tony', 'time': 20}, '4871231234': {'name': 'Rebecca', 'time': 10}}

# dictionary to store pickup/delivery prices
DeliveryOptions = {'pickup': 0.00, 'delivery': 3.00, 'pick up': 0.00, 'pick-up': 0.00,
                   'take out': 0.00, 'takeout': 0.00, 'takeaway': 0.00, 'take-out': 0.00,
                   'take-away': 0.00, 'take away': 0.00, 'carry out': 0.00, 'carryout': 0.00,
                   'carry-out': 0.00}

#stores variable and corresponding question
questions = {'pizza_type': 'What pizza would you like? ', 'pizza_size':'What size pizza would you like? We have small, medium, and large. ', 'crust_type': 'What type of crust would you like? We have regular, deep dish, thin, and gluten free. ',
              'delivery_option': 'Would you like delivery or pickup in store? ', 'address': 'What is your address? ', 'phone_number': 'What\'s your phone number? ', 'name': 'What\'s your name? '}

menu_items = ['hawaiian', 'meat lovers', '4 cheese', 'pepperoni', 'veggie supreme', 'vegan',
              'mozzarella', 'cheddar', 'swiss', 'provolone', 'pineapple', 'green peppers',
              'red onions', 'mushrooms', 'black olives', 'pepperoni', 'ham', 'bacon', 'sausage',
              'thin', 'regular', 'deep dish', 'gluten free', 'stuffed', 'small', 'medium', 'large', 
              '10"', '12"', '14"', 'garlic knots', 'mozzarella sticks', 'green salad', 'caesar salad',
              'cola', 'root beer', 'orange soda', 'lemon soda', 'mineral water', 'ginger ale',
              'pick up', 'pickup', 'pick-up', 'delivery', 'take out', 'takeout', 'takeaway',
              'take-out', 'take-away', 'take away', 'carry out', 'carryout', 'carry-out']

one_word_toppings = ['mozzarella', 'cheddar', 'swiss', 'provolone', 'pineapple', 'mushrooms', 
                     'pepperoni', 'ham', 'bacon', 'sausage']

del_opts = ['pick up', 'pickup', 'pick-up', 'delivery', 'take out', 'takeout', 'takeaway',
              'take-out', 'take-away', 'take away', 'carry out', 'carryout', 'carry-out']
# why need this and the menu_items above? This one has printout names?
all_menu_items = ["hawaiian pizza", "meat lovers pizza", "4 cheese", "pepperoni pizza", 
                  "veggie supreme pizza", "vegan pizza", "mozzarella", "cheddar", "swiss", "provolone",
                  "pineapple", "green peppers", "red onions", "mushrooms", "black olives", "pepperoni",
                  "ham", "bacon", "sausage", "thin crust", "regular crust", "deep dish crust",
                  "gluten free crust", "cheese-stuffed crust", "green salad", "caesar salad",
                  "cola", "root beer", "orange soda", "lemon soda", "mineral water", "ginger ale",
                  "garlic knots", "mozzarella sticks"]

pizza_num_mapping = {'first': 0, 'second': 1, 'third': 2, 'fourth': 3, 'fifth': 4, 'sixth': 5, 'seventh': 6,
                     'eighth': 7, 'ninth': 8, 'tenth': 9, 'eleventh': 10, 'twelfth': 11, 'thirteenth': 12,
                     'fourteenth': 13, 'fifteenth': 14, 'sixteenth': 15, 'seventeenth': 16, 'eighteenth': 17,
                     'nineteenth': 18, 'twentieth': 19}

size_choices_mapping = {'first': 'small', 'second': 'medium', 'third': 'large', 'last': 'large'}

crust_choices_mapping = {'first': 'regular', 'second': 'thin', 'third': 'deep dish',
                         'fourth': 'gluten free', 'last': 'gluten free'}

delivery_choices_mapping = {'first': 'pickup', 'second': 'delivery', 'last': 'delivery'}

wrong_names = {'vegetarian': 'veggie supreme', 'the one with the meat': 'meat lovers',
               'the meat one': 'meat lovers', 'no dairy no meat': 'vegan', 'no meat no dairy': 'vegan',
               'a cheese': '4 cheese', 'the cheese': '4 cheese', 'veggie delight': 'veggie supreme', 
               'the one with pineapple': 'hawaiian', 'the pineapple one': 'hawaiian',
               'the one with no gluten': 'gluten free', 'the kind with no gluten': 'gluten free', 
               'cheesy crust': 'stuffed', 'cheese stuffed': 'stuffed'}

example_names = ['sara', 'tom', 'jacob', 'katie']
example_addresses = ['123 elm street', '1234 main street', '321 oak street']

#file: slot_questions.py **************************************************************************************************************

"""
For each slot, some questions that the system can randomly choose from to ask what the user wants to fill that slot
"""

pizza_type = ["What kind of pizza would you like?\n", "What pizza would you like?\n",
              "What kind of pizza can I get you?\n"]

pizza_size = ["What size pizza would you like?\n", "What size pizza would you like? We have small, medium, and large.\n",
              "Do you want your pizza to be small, medium, or large?\n"]

crust_type = ["What type of crust would you like? We have regular, thin, deep dish, and gluten free.\n", "What type of crust would you like?\n",
              "Would you like regular, thin, deep dish, or gluten free crust?\n"]

delivery_option = ["Would you like pickup or delivery?\n", "Pickup or delivery?\n", "Is this order for pickup or delivery?\n"]

address = ["What's your address?\n", "What address should we deliver to?\n"]

phone_number = ["What's your phone number?\n", "What's a good phone number for your order?\n"]

name = ["What's your name? It helps me if you say \"My name is\" before your name.\n", "What name should I call you? It helps me if you say \"My name is\" before your name.\n"]

drinks = ["Would you like any drinks to go with your order?\n", "Any drinks?\n", "Any drinks to go with your order?\n"]

sides = ["Would you like any sides?\n", "Any sides today?\n", "Any sides?\n"]


#confirm = ["Okay, great!", "Sounds good!", "Sounds great!", "Okay, got it."]

#deny = ["Sorry, we don't have that on our menu"]

""" Imports, mapping all the different intents, get annotated data?"""

#crf.py ***********************************************************************************************************************************


#Mapping different groups slot tags to one set of slot tags
mapping = {
    'PIZZA' : ['Pizza'],
    'COUNT' : ['Pizza quantity', 'quantity', 'pizza_quantity', 'quant', 'num_pizzas'],
    'PIZZA_NUMBER' : ['Pizza number', 'pizza_number'],
    'PIZZA_TYPE' : ['Pizza type', 'specialty', 'pizza_type', 'specialty_pizza', 'special', 'Pizza_type', 'Add-to-pizza-type-list'],
    'TOPPING' : ['Pizza topping', 'topping', 'modification', 'topp', 'toppings', 'Toppings', 'Add-to-toppings-list'],
    'SIZE' : ['Pizza size', 'size', 'pizza_size', 'Size', 'Add-to-size-list'],
    'CRUST' : ['Pizza crust', 'crust', 'crust_type', 'pizza_crust', 'Crust', 'Add-to-crust-list'],
    'SIDE' : ['Side order', 'side', 'Sides'],
    'DRINK' : ['drink', 'beverage', 'drinks'],
    'COST' : ['Pizza cost', 'cost', 'price'],
    'PHONENUMBER' : ['Phone number', 'phone_number', 'phone', 'number', 'phone_num', 'customer_number'],
    'ADDRESS' : ['Address', 'address', '(street_)address', 'customer_address'],
    'NAME' : ['Order name', 'name', 'customer_name', 'Name'],
    'ORDER_TYPE' : ['Delivery method', 'order_type', 'delivery_option', 'delivery', 'modality', 'service', 'delivery_type', 'method_of_attainment', 'Delivery'],
    'SLOT_ATTRIBUTE' : ['Information type', 'feature', 'Topic'],
    'PICKUPADDRESS' : ['Pickup location', 'store_location', 'pickup_location', 'location', 'store', 'branch_name', 'Location'],
    'PAYMENTTYPE' : ['Payment type', 'payment_option', 'payment', 'payment_method'],
    'CARD' : ['Card type', 'payment_option', 'payment', 'card', 'payment_method', 'card_type'],
    'CARDNUMBER' : ['Card number', 'payment_option', 'numeric', 'card_num', 'card_number'],
    'CARDEXP' : ['Card expiration', 'payment_option', 'card_exp', 'expiration_date'],
    'CARDCCV' : ['Card CCV', 'payment_option', 'numeric', 'card_code', 'security_code'],
    'ORDERTIME' : ['Order time', 'time'],
    'ALLERGY' : ['Allergy'],
    'TIP' : ['Tip amount'],
    'ITEMS' : ['items'],
    'REORDER' : ['reorder', 'reorder_'],
    'STATUS' : ['query_status'],
    'CHANGE' : ['change_current_'],
    'FOLLOWUP' : ['follow_up'],
    'TOPPING_CLASS' : ['topping_type'],
    'ORDER_MODIFIER' : ['alt', 'property'],
    'MODIFICATIONS' : ['mod'],
    'CONFIRM' : ['confirm'],
    'GREETING' : ['greeting'],
    'QUESTION_CLASS' : ['request_type'],
    'QUESTION_CONT' : ['request_content', 'Item[1..n]'],
    'PAYMENT_TIME' : ['Payment_time']
}


#remove periods, commas, and question marks
def remove_punct(sent):
  return sent.replace('.', '').replace(',', '').replace('?', '')

#create feature dicts from words
def word2features(word):
  feats = {'word':word}
  if word.count('<') == 2:
    first = word.find('<')
    word = word[first:] #remove any letters before first < (for if there is typo in data)
    last = word.rfind('>')
    word = word[:last] #remove any letters after last > (for if there is typo in data)

    word_and_tags = word.replace(">", " ").replace("<", " ").replace("/", " ")
    if len(word_and_tags.split()) == 3:
      tag, w, extraTag = word_and_tags.split()
      feats = {'word':w}
  elif word.count('<') == 1:
    if word[0] == '<':
      if len(word.split('>')) == 2:
        tag, w = word.split('>')
        feats = {'word':w}
    elif word[len(word) - 1] == '>':
      if len(word.split('<')) == 2:
        w, tag = word.split('<')
        feats = {'word':w}
  elif word.count('<') == 0:
    feats = {'word':word}
  return feats

#for each word in sentence, call words2features
def sent2features(sent):
  return [word2features(word) for word in sent.split()]

#get slot annotations from words
def word2labels(word):
  if word.count('<') == 2:
    first = word.find('<')
    word = word[first:] #remove any letters before first < (for if there is typo in data)
    last = word.rfind('>')
    word = word[:last] #remove any letters after last > (for if there is typo in data)

    word_and_tags = word.replace(">", " ").replace("<", " ").replace("/", " ")
    if len(word_and_tags.split()) == 3:
      tag, w, extraTag = word_and_tags.split()
      for m in mapping:
        if tag in mapping[m]:
          return m
    else:
      return 'O'
  elif word.count('<') == 1:
    if word[0] == '<':
      if len(word.split('>')) == 2:
        tag, w = word.split('>')
        tag = tag.replace('<', '').replace('/', '')
        for m in mapping:
          if tag in mapping[m]:
            return m
      else:
        return 'O'
    elif word[len(word) - 1] == '>':
      if len(word.split('<')) == 2:
        w, tag = word.split('<')
        tag = tag.replace('>', '').replace('/', '')
        for m in mapping:
          if tag in mapping[m]:
            return m
      else:
        return 'O'
  elif word.count('<') == 0:
    return 'O'
  return 'O'

#for each word in sentence, call words2labels
def sent2labels(sent):
  return [word2labels(word) for word in sent.split()]


#get data from files and make/train model
def train_model(dirname):
  data_list = []
  print('HERES DIRNAME: ', dirname)
  # have to change how to navigate folders in Drive
  for folder in os.listdir(dirname):
    print("HERES A FOLDER: ", folder)
    new_dir = dirname + '/' + str(folder)
    print('HERES NEW DIR: ', new_dir)
    files = os.listdir(new_dir)
    print('FILES: ', files)
    for file in files:
      with open(os.path.join(new_dir,file), 'r') as f:
        data = f.readlines()
      for line in data[1:]:
        data_list.append(line)

  #use functions to prepare training and testing data
  x_train = [sent2features(remove_punct(s)) for s in data_list]
  y_train = [sent2labels(remove_punct(s)) for s in data_list]

  #make model
  crf = sklearn_crfsuite.CRF(
      algorithm='lbfgs',
      c1=0.1,
      c2=0.1,
      max_iterations=100,
      all_possible_transitions=True
  )

  #fit model
  crf.fit(x_train, y_train)

  #get list of labels used
  labels = list(crf.classes_)
  labels.remove('O')

  #return model
  return crf

"""
definition of class SemanticFrame and helper class Pizza **********************************************************************************
"""

# added this in instead of importing semantic_frame_class 7/21/21
class Pizza:
  def __init__(self):
    self.info = {'pizza_type': '', 'pizza_size': '', 'crust_type': '', 'toppings': []}

class SemanticFrame:
  def __init__(self):
        self.last_system_utterance = ''
        self.pizza_num = 0
        self.Intent = None
        self.Slots = {'pizzas' : [], 'sides':[], 'drinks':[],
              'delivery_option': '', 'phone_number': '', 'address': '', 'name':''}

# just nlg_question_order function from main_funcs? ****************************************************************************************

def nlg_question_order(semantic_frame, crf):
  # test each slot to see if empty so it updates right after
  full = 0
  while full == 0:
    success = 0
    # loops through each pizza in the SF
    for i in range(len(semantic_frame.Slots['pizzas'])):
      # checks if that pizza's pizza_type slot is empty or not
      if semantic_frame.Slots["pizzas"][i].info['pizza_type'] == '':
        success = 1
        # if only 1 pizza in the frame, don't need to specify which pizza!
        if len(semantic_frame.Slots['pizzas']) == 1:
          # randomly pick one of the versions of the pizza type question
          semantic_frame.last_system_utterance = random.choice(pizza_type)
        else:
          # specify which pizza (first? fifth?) when asking
          semantic_frame.last_system_utterance = "What kind of pizza do you want your " + num_to_word[i] + " pizza to be?\n"
          semantic_frame.pizza_num = i
        inputStr = input(semantic_frame.last_system_utterance)
        # parse input string and add to SF
        semantic_frame = nlu_parse(inputStr, semantic_frame, crf)
        break
      elif semantic_frame.Slots["pizzas"][i].info['pizza_size'] == '':
        success = 1
        if len(semantic_frame.Slots['pizzas']) == 1:
          semantic_frame.last_system_utterance = random.choice(pizza_size)
        else:
          semantic_frame.last_system_utterance = "What size do you want your " + num_to_word[i] + " pizza to be?\n"
          semantic_frame.pizza_num = i
        inputStr = input(semantic_frame.last_system_utterance)
        semantic_frame = nlu_parse(inputStr, semantic_frame, crf)
        break
      elif semantic_frame.Slots['pizzas'][i].info["crust_type"] == '':
        success = 1
        if len(semantic_frame.Slots['pizzas']) == 1:
          semantic_frame.last_system_utterance = random.choice(crust_type)
        else:
          semantic_frame.last_system_utterance = "What kind of crust do you want your " + num_to_word[i] + " pizza to have?\n"
          semantic_frame.pizza_num = i
        inputStr = input(semantic_frame.last_system_utterance)
        semantic_frame = nlu_parse(inputStr, semantic_frame, crf)
        break
    # removed sq. etc - for lists of options from slot_questions function
    if success == 0:
      if semantic_frame.Slots["delivery_option"] == '':
        semantic_frame.last_system_utterance = random.choice(delivery_option)
        inputStr = input(semantic_frame.last_system_utterance)
        semantic_frame = nlu_parse(inputStr, semantic_frame, crf)
      elif semantic_frame.Slots['phone_number'] == '':
        semantic_frame.last_system_utterance = random.choice(phone_number)
        inputStr = input(semantic_frame.last_system_utterance)
        semantic_frame = nlu_parse(inputStr, semantic_frame, crf)
      elif semantic_frame.Slots['name'] == '':
        semantic_frame.last_system_utterance = random.choice(name)
        inputStr = input(semantic_frame.last_system_utterance)
        semantic_frame = nlu_parse(inputStr, semantic_frame, crf)
      elif semantic_frame.Slots["delivery_option"]=='delivery' and semantic_frame.Slots["address"]=='':
        semantic_frame.last_system_utterance = random.choice(address)
        inputStr = input(semantic_frame.last_system_utterance)
        semantic_frame = nlu_parse(inputStr, semantic_frame, crf)
    
    # check to see if every field full, if one is empty stop and resume while loop
    full = 1
    # check to see if it's the case where delivery is pickup and so address is empty
    for item in semantic_frame.Slots:
      if item == 'pizzas':
        for pizza in semantic_frame.Slots['pizzas']:
          for inf in pizza.info:
            if inf == '':
              full = 0
      else:
        # if field is currently empty
        if semantic_frame.Slots[item] == '':
          # check to see if it's just address being empty and we can ignore it
          if item=='address' and semantic_frame.Slots['delivery_option']=='pickup':
            pass
          # or its address empty but we need to fill it here
          elif item=='address' and semantic_frame.Slots['delivery_option']=='delivery':
            full = 0
          # or its not address and it needs to be filled
          else:
            full = 0

#*******************************************************************************************************************************************
# chitchat

#import main_funcs


"""
handle non-order related dialogue
"""
choice = [0, 1]
generic = ["That's really cool! ", "How interesting! ", "Me too! "] #, "Interesting, why's that? ", "Tell me more about that. "]

question_words = ["does", "who", "what", "where", "when", "why", "which", "how", "do", "whose", "whats"]

Chit_chat_questions = {'how do you feel about pineapple on pizza' : "Great! We have Hawaiian Pizza at Tony's. ",
                       "who has better pizza tonys or marios" : "Tony's! ",
                       "what kind of person doesnt like pizza" : "A weirdough! ",
                       "youre a great pizza ordering system" : "Thanks! ",
                       "whats your favorite pizza": "4 Cheese with extra cheese! ",
                       "whats your name": "P.I.Z.Z.A: Pleasant Interactive Zealous Zany Assistant! ",
                       "write me a poem": "Pizza: A Haiku \nI love you pizza \nGive me all the toppings please \nDon't leave me pizza \n",
                       "what time is it": "Pizza Time! ",
                       "how old are you": "That's not a polite question. "}

def chitchat(utterance, semantic_frame, crf):
  if utterance in Chit_chat_questions:
    system_utterance = Chit_chat_questions[utterance]
  else:
    ch = random.choice(choice)
    tokens = utterance.split()
    if ch == 0 or tokens[0] in question_words:
      tokens = utterance.split()
      for i in range(len(tokens)):
        if tokens[i] == 'i':
          tokens[i] = 'TEMP'
      for i in range(len(tokens)):
        if tokens[i] == 'me':
          tokens[i] = 'TEMP'
      for i in range(len(tokens)):
        if tokens[i] == 'you':
          tokens[i] = 'I'
      for i in range(len(tokens)):
        if tokens[i] == 'TEMP':
          tokens[i] = 'you'


      for i in range(len(tokens)):
        if tokens[i] == 'myself':
          tokens[i] = 'TEMP'
      for i in range(len(tokens)):
        if tokens[i] == 'yourself':
          tokens[i] = 'myself'
      for i in range(len(tokens)):
        if tokens[i] == 'TEMP':
          tokens[i] = 'yourself'
      

      for i in range(len(tokens)):
        if tokens[i] == 'im':
          tokens[i] = 'TEMP'
      for i in range(len(tokens)):
        if tokens[i] == 'youre':
          tokens[i] = "I'm"
      for i in range(len(tokens)):
        if tokens[i] == 'TEMP':
          tokens[i] = "you're"



      for i in range(len(tokens)):
        if tokens[i] == 'your':
          tokens[i] = 'TEMP'
      for i in range(len(tokens)):
        if tokens[i] == 'my':
          tokens[i] = 'your'
      for i in range(len(tokens)):
        if tokens[i] == 'TEMP':
          tokens[i] = 'my'


      for i in range(len(tokens)):
        if tokens[i] == 'yours':
          tokens[i] = 'TEMP'
      for i in range(len(tokens)):
        if tokens[i] == 'mine':
          tokens[i] = 'yours'
      for i in range(len(tokens)):
        if tokens[i] == 'TEMP':
          tokens[i] = 'mine'

      system_utterance = ' '.join(tokens)
      if tokens[0] in question_words:
          system_utterance = system_utterance.capitalize() + "? Hmm, I'm not sure. " 
      else:
          system_utterance = system_utterance.capitalize() + "? " + random.choice(generic)

    else:
      system_utterance = random.choice(generic)
  
  print(system_utterance, end = "")
  nlg_question_order(semantic_frame, crf)
#**************************************************************************************************************************************************
#request.py

"""
For menu questions and recommendations

For menu questions: determine what the user is asking about and choose the right response
For general recommendations: choose recommendation from generic list
For specific recommendations (allergies, vegan, etc.): find a keyword (vegan, gluten free, etc.) and choose a recommendation based on that 
"""

general_recommendations = ["Our most popular order is a large Pepperoni Pizza on regular crust. ", 
                   "We're particularly proud of our Hawaiian Pizza! ",
                   "Our customers love our 4 Cheese Pizza! "]

#helper function to make a list into a string to answer the user
def make_str(lst):
  s = ''
  for l in lst[:-1]:
    s = s + l + ", "
  s = s + 'and ' + lst[-1]
  return s

#determine what kind of question a user is asking and call that type of question's specific function
def customer_question(tokens, semantic_frame, crf):
  success = 0
  size_words = ['big', 'size', 'large']
  price_words = ['cost', 'price', 'much']
  topping_words = ['whats on', 'toppings on', 'what toppings are on', 'what toppings']
  what_kind_words = ['what toppings do ', 'what kind', 'what kinds of pizzas', 'what kind of pizzas', 'what sizes', 'are there', 'all your', 'specialty pizzas', 'what are your pizzas', 'delivery options', 'options for delivery']
  menu_item_exist_words = ['do you have', "does your menu", 'on your menu']
  recommend_words = ['recommend', 'suggest', "whats good here", "recommendation", "recommendations"]
  tokens_str = " ".join(tokens)

  for words in recommend_words:
    if words in tokens_str:
      success = 1
      recommend(tokens)
      nlg_question_order(semantic_frame, crf)
      return
  for words in topping_words:
    if words in tokens_str:
      success = 1
      topping_questions(tokens)
      nlg_question_order(semantic_frame, crf)
      return
  for words in what_kind_words:
    if words in tokens_str:
      success = 1
      what_kind_questions(tokens)
      nlg_question_order(semantic_frame, crf)
      return
  for words in menu_item_exist_words:
    if words in tokens_str:
      success = 1
      menu_item_exist_question(tokens)
      nlg_question_order(semantic_frame, crf)
      return

  for token in tokens:
    # only handles questions where price comes before size
    if token in size_words:
      success = 1
      size_questions(tokens)
      nlg_question_order(semantic_frame, crf)
      return
    elif token in price_words:
      success = 1
      price_questions(tokens)
      nlg_question_order(semantic_frame, crf)
      return
    elif token in topping_words:
      success = 1
      topping_questions(tokens)
      nlg_question_order(semantic_frame, crf)
      return
    # 
    """ 
    elif token == 'recommend':
      recommend(tokens)
      mf.nlg_question_order(semantic_frame, crf)
    """

  if success == 0:
    ut = " ".join(tokens)
    # changed from chitchat.chitchat
    chitchat(ut, semantic_frame, crf)

#What size is a menu item?
def size_questions(tokens):
  dimensions = ' '
  size = ''
  for token in tokens:
    if token in PizzaSizes:
      dimensions = PizzaSizes[token]
      size = token
      print("The " + size + " is " + dimensions + ".", end = " ")
  if dimensions == ' ':
    while dimensions == ' ':
      str = input("Sorry, what size did you want to know the dimensions of? We have small, medium, and large.\n")
      tokens = re.sub(r'[^\w\s]', '', str).lower().strip().split()
      for token in tokens:
        if token in PizzaSizes:
          dimensions = PizzaSizes[token]
          size = token
          print("The " + size + " is " + dimensions + ".", end = " ")

#How much does something cost?
def price_questions(tokens):
  tokens_str = " ".join(tokens)
  told_price = 0
  for option in DeliveryOptions:
    if option in tokens_str:
      print("The price of " + option + " is $" + str(DeliveryOptions[option]) + ".", end = " ")
      told_price = 1
  for size in PizzaCrusts:
    if size in tokens_str:
      for crust in pizza_crusts:
        if crust in tokens_str:
          formatted_price = "{:.2f}".format(PizzaCrusts[size][crust])
          print("The price of a " + size + " pizza with " + crust + " crust is $" + str(formatted_price) + ".", end = " ")
          told_price = 1
  for drink in Drinks:
    if drink in tokens_str:
      print("The price of " + drink + " is $" + str(Drinks[drink]) + ".", end = " ")
      told_price = 1
  for side in Sides:
    if side in tokens_str:
      print("The price of " + side + " is $" + str(Sides[side]) + ".", end = " ")
      told_price = 1
  for topping in Toppings:
    if topping in tokens_str:
      print("The price of " + topping + " is $" + str(Toppings[topping]) + ".", end = " ")
      told_price = 1
  if told_price == 0:
    new_str = input("Sorry, what did you want the price of? If you want the price of a pizza, we have to know the size and crust type.\n")
    new_tokens = re.sub(r'[^\w\s]', '', new_str).lower().strip().split()
    price_questions(new_tokens)

#What toppings are on the specialty pizza?
def topping_questions(tokens):
  tokens_str = " ".join(tokens)
  toppings = [' ']
  pizza = ''
  for sp in SpecialtyPizzas:
    if sp in tokens_str:
      toppings = SpecialtyPizzas[sp]
      pizza = sp
      topping_str = ''
      for topping in toppings[:-1]:
        topping_str = topping_str + topping + ", "
      topping_str = topping_str + "and " + toppings[-1]
      print("The toppings on the " + pizza + " are " + topping_str + ".", end = " ")
  if toppings == [' ']:
    while toppings == [' ']:
      str = input("Sorry, what kind of specialty pizza did you want to know the toppings of? We have Hawaiian, Meat Lovers, 4 Cheese, Pepperoni, Veggie Supreme, and Vegan.\n")
      str = re.sub(r'[^\w\s]', '', str).lower().strip()
      for sp in SpecialtyPizzas:
        if sp in str:
          toppings = SpecialtyPizzas[sp]
          pizza = sp
          topping_str = ''
          for topping in toppings[:-1]:
            topping_str = topping_str + topping + ", "
          topping_str = topping_str + "and " + toppings[-1]
          print("The toppings on the " + pizza + " are " + topping_str + ".", end = " ")


#What kind of pizzas do you have, what sizes do you have, what toppings do you have, etc.
def what_kind_questions(tokens):
  tokens_str = " ".join(tokens)
  found = 0
  if "topping" in tokens_str or "toppings" in tokens_str:
    print("We have mozzarella, cheddar, swiss, prvolone, pineapple, green peppers, red onions, mushrooms, black olives, pepperoni, ham, bacon, and sausage.", end = " ")
    found = 1
  elif "pizza" in tokens_str or "pizzas" in tokens_str:
    print("We have Hawaiian, Meat Lovers, 4 Cheese, Pepperoni, Veggie Supreme, and Vegan.", end = " ")
    found = 1
  elif "size" in tokens_str or "sizes" in tokens_str:
    print("We have small, medium, and large" + ". ", end = "")
    found = 1
  elif "crust" in tokens_str or "crusts" in tokens_str:
    print("We have regular, thin, deep dish, cheese-stuffed, and gluten free crusts. ", end = "")
    found = 1
  elif "delivery" in tokens_str or "pickup" in tokens_str:
    print("We have pickup and delivery.", end = " ")
    found = 1
  if found == 0:
    print("Sorry, I didn't understand the question.", end = " ")

#Do you have blank on your menu?
def menu_item_exist_question(tokens):
  tokens_str = " ".join(tokens)
  found = 0
  s = ''
  for item in all_menu_items:
    if item in tokens_str:
      found += 1
      if item[-1] == 's': #if plural
        s = s + item + " are on the menu. "
      else: #if singular
        s = s + item + " is on the menu. "
  if found == 0:
    s = "Sorry, that's not on the menu. "
  #s = s.strip() #remove trailing whitespace
  print(s, end = "")


#What do you recommend?
def recommend(tokens):
  tokens_str = " ".join(tokens)
  if 'vegetarian' in tokens_str:
    print("I recommend our Veggie Supreme Pizza and our Caesar Salad.", end = " ")
  elif 'vegan' in tokens_str:
    print("I recommend our Vegan Pizza and our Green Salad.", end = " ")
  elif 'gluten' in tokens_str:
    print("I recommend our Pepporoni Pizza on gluten free crust.", end = " ")
  elif 'topping' in tokens_str:
    print("I recommend the black olives.", end = " ")
  elif 'side' in tokens_str:
    print("I recommend the cheese sticks.", end = " ")
  elif 'drink' in tokens_str:
    print("I recommend the orange soda.", end = " ")
  else:
    print(random.choice(general_recommendations), end = "")

#*******************************************************************************************************************************************************

#main_funcs.py

num_to_word = {0: 'first', 1: 'second', 2: 'third', 3: 'fourth', 4: 'fifth', 5: 'sixth',
               6: 'seventh', 7: 'eighth', 8: 'ninth', 9: 'tenth', 10: 'eleventh', 11: 'twelfth',
               12: 'thirteenth', 13: 'fourteenth', 14: 'fifteenth', 15: 'sixteenth',
               16: 'seventeenth', 17: 'eighteenth', 18: 'nineteenth', 19: 'twentieth'}

# handles each pizza - it has a slot in Slots, gets other info about it from input, understands its number when ordering multiple at once?
def translate_pred(pred, sent, sf): #sf is semantic frame
  found = 0
  s = remove_punct(sent)
  tokens = s.split()
  p = pred[0]
  # go through each token and sets number of total pizzas in SF to that number if it's greater than the current num
  for i in range(0, len(tokens)):
    if tokens[i] in pizza_num_mapping: # and tokens[i] != 'first':
      # if input number ('seventh' for example) is in the above dict
      sf.pizza_num = pizza_num_mapping[tokens[i]]
      # if input num is greater than or equal to the current number of pizza slots in the SF:
      if sf.pizza_num >= len(sf.Slots['pizzas']):
        # if only greater than, set the pizza num in the SF to the length of the pizza slots??? Huh??? Wouldnt it be other way around?
        if sf.pizza_num > len(sf.Slots['pizzas']):
          sf.pizza_num = len(sf.Slots['pizzas'])
        # only make new pizza object if input num is greater or equal to pizza slot amt
        Pi = Pizza()
        #print(sf.Slots['pizzas'])
        # then just add new pizza on at end of slots?
        sf.Slots['pizzas'].append(Pi)
    # still looping through tokens--find slots/info for that pizza
    # but why keep pizza num the same when you've added one more than the slot length??
    if p[i] == 'PIZZA_TYPE':
      sf.Slots['pizzas'][sf.pizza_num].info['pizza_type'] = tokens[i]
      found = 1
    elif p[i] == 'SIZE':
      sf.Slots['pizzas'][sf.pizza_num].info['pizza_size'] = tokens[i]
      found = 1
    elif p[i] == 'CRUST':
      sf.Slots['pizzas'][sf.pizza_num].info['crust_type'] = tokens[i]
      found = 1
    elif p[i] == 'SIDE':
      sf.Slots['sides'].append(tokens[i])
      found = 1
    elif p[i] == 'DRINK':
      sf.Slots['drinks'].append(tokens[i])
      found = 1
    elif p[i] == 'TOPPING':
      sf.Slots['pizzas'][sf.pizza_num].info['toppings'].append(tokens[i])
      found = 1
    elif p[i] == 'ORDER_TYPE': #delivery/pickup
      sf.Slots['delivery_option'] = tokens[i]
      found = 1
    elif p[i] == 'PHONENUMBER':
      sf.Slots['phone_number'] = tokens[i]
      found = 1
    # would each pizza realistically have its own address? Can't I just make them all one address?
    elif p[i] == 'ADDRESS':
      sf.Slots['address'] = tokens[i]
      found = 1
    elif p[i] == 'NAME':
      sf.Slots['name'] = tokens[i]
      found = 1
  
  return sf, found #return semantic frame

# REMOVED NLG_QUESTION_ORDER FROM HERE!!!!!!!!!!!!!!!
"""
def spell_check(string):
  spell = SpellChecker()
  tokens = string.split()
  misspelled = spell.unknown(tokens)
  new_string = ""
  for word in tokens:
    if word not in misspelled:
      new_string += word + " "
    else:
      new_string += spell.correction(word) + " "
  new_string = new_string.strip()
  return new_string
"""

# goes through each word of input
def nlu_parse(inputStr, SemanticFrame, crf):
  # remove punctuation
  inputStr = re.sub(r'[^\w\s]', '', inputStr)
  orig = inputStr #with original capitalization
  inputStr = inputStr.lower()
  #inputStr = spell_check(inputStr)

  tokens = inputStr.lower().strip().split() #lowercase and split by whitespace

  # variable for whether or not input is a request
  request = 0
  tokens_detected = 0

  if inputStr in Chit_chat_questions:
    chitchat(inputStr, SemanticFrame, crf)
    tokens_detected += 1
  elif 'repeat' in inputStr:
    tokens_detected += 1
    nlg_question_order(SemanticFrame, crf)
  elif 'cancel' in inputStr:
    tokens_detected += 1
    print('Sorry to see you go, have a great night!')
    sys.exit()
  elif 'start over' in inputStr:
    #clear semantic frame
    tokens_detected += 1
    SemanticFrame.Slots['pizzas'] = []
    p = Pizza()
    SemanticFrame.Slots['pizzas'].append(p) #add blank pizza to list of pizzas in SemanticFrame.slots
    SemanticFrame.Slots['delivery_option'] = ''
    SemanticFrame.Slots['name'] = ''
    SemanticFrame.Slots['address'] = ''
    SemanticFrame.Slots['phone_number'] = ''
    SemanticFrame.Slots['sides'] = []
    SemanticFrame.Slots['drinks'] = []

    """
    #single pizza order version
    SemanticFrame.Slots['pizza_type'] = ''
    SemanticFrame.Slots['pizza_size'] = ''
    SemanticFrame.Slots['crust_type'] = ''
    SemanticFrame.Slots['delivery_option'] = ''
    SemanticFrame.Slots['name'] = ''
    SemanticFrame.Slots['address'] = ''
    SemanticFrame.Slots['phone_number'] = ''
    SemanticFrame.Slots['toppings'] = []
    SemanticFrame.Slots['sides'] = []
    SemanticFrame.Slots['drinks'] = []
    """
    nlg_question_order(SemanticFrame, crf)


  # lowercase and split by whitespace
  #tokens = inputStr.lower().strip().split()

  # detecting question words, order query and REQUEST
  elif tokens[0] == "what" or tokens[0] == "how" or tokens[0]=='whats' or tokens[0] == 'do':
    request = 1
    # put question they asked into function
    customer_question(tokens, SemanticFrame, crf)
    SemanticFrame.Intent = "REQUEST"
    tokens_detected += 1
  elif 'check' in inputStr or 'status' in inputStr or 'when will' in inputStr or 'when my' in inputStr or 'get here' in inputStr or 'be here' in inputStr or 'be delivered' in inputStr or 'be ready' in inputStr or 'what time' in inputStr:
    tokens_detected += 1
    answer = input('Okay. What is your phone number? ')
    answer = re.sub(r'[^\w\s]', '', answer)
    # if phone number found, retrieve order info from dict
    if current_orders.get(answer):
      print('Great, thank you. I see there is an order currently on its way to', current_orders[answer].get('name') + '. You can expect it in about', current_orders[answer].get('time'), 'minutes. Thanks for ordering from Tony\'s and have a great night!')
      sys.exit()
    else:
      print('Sorry, we didn\'t recognize that phone number. ', end = '') # If you think this is a mistake, type \'status\' and try again. Otherwise continue by typing which pizza you want: ')
      nlg_question_order(SemanticFrame, crf)
  elif 'reorder' in inputStr or 'get my favorite' in inputStr or 'order my favorite' in inputStr or 'order my preferred' in inputStr or 'get my preferred' in inputStr or 'my usual' in inputStr:
    tokens_detected +=1
    answer = input('Okay. What is your phone number? ')
    answer = re.sub(r'[^\w\s]', '', answer)
    # if phone number is found, set semantic frame to their info
    if regulars.get(answer):
      #pp = Pizza()
      #SemanticFrame.Slots['pizzas'].append(pp)
      SemanticFrame.Slots['pizzas'][0].info['pizza_type'] = regulars[answer]['pizza_type']
      SemanticFrame.Slots['pizzas'][0].info['pizza_size'] = regulars[answer]['pizza_size']
      SemanticFrame.Slots['pizzas'][0].info['crust_type'] = regulars[answer]['crust_type']
      SemanticFrame.Slots['delivery_option'] = regulars[answer]['delivery_option']
      SemanticFrame.Slots['phone_number'] = answer
      SemanticFrame.Slots['name'] = regulars[answer]['name']
      SemanticFrame.Slots['address'] = regulars[answer]['address']
    else:
      print('Sorry, we didn\'t recognize that phone number. ', end = '') # If you think this is a mistake, type \'reorder\' and try again. Otherwise continue by typing which pizza you want: ')
      nlg_question_order(SemanticFrame, crf)

  else:

    success = 0

    if SemanticFrame.last_system_utterance in referring_questions:
      success = referring_expressions(inputStr, SemanticFrame, crf)
      tokens_detected += 1

    #print(success)
    if success == 0:
      #use model to make predictions
      #txt = orig
      txt = inputStr
      test = [sent2features(remove_punct(s)) for s in [txt]]
      pred = crf.predict(test)
      SemanticFrame, found  = translate_pred(pred, txt, SemanticFrame)
      if found == 1:
        tokens_detected = 1

      multiple_orders = [inputStr]
      for number in pizza_num_mapping:
        if number in inputStr:
          index = multiple_orders[len(multiple_orders) - 1].find(number)
          oldstring = multiple_orders[len(multiple_orders) - 1][:index]
          newstring = multiple_orders[len(multiple_orders) - 1][index:]
          multiple_orders.remove(multiple_orders[len(multiple_orders) - 1])
          multiple_orders.append(oldstring)
          multiple_orders.append(newstring)

      multiple_orders[:] = [x for x in multiple_orders if x] #remove empty string
      #print(multiple_orders)

      for order in multiple_orders:
        tokens = order.split()
        if tokens[0] in pizza_num_mapping:
          SemanticFrame.pizza_num = pizza_num_mapping[tokens[0]]
          if SemanticFrame.pizza_num >= len(SemanticFrame.Slots['pizzas']):
            if SemanticFrame.pizza_num > len(SemanticFrame.Slots['pizzas']):
              SemanticFrame.pizza_num = len(SemanticFrame.Slots['pizzas'])
            Pi = Pizza()
            #print(sf.Slots['pizzas'])
            SemanticFrame.Slots['pizzas'].append(Pi)


        # look for two-word nouns like deep dish, 4 cheese, gluten free, veggie supreme, and meat lovers
        if 'garlic knots' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["sides"].append('garlic knots')
        if 'mozzarella sticks' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["sides"].append('mozzarella sticks')
        if 'green salad' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["sides"].append('green salad')
        if 'caesar salad' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["sides"].append('caesar salad')

        if 'green peppers' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["pizzas"][SemanticFrame.pizza_num].info["toppings"].append('green peppers')
        if 'red onions' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["pizzas"][SemanticFrame.pizza_num].info['toppings'].append('red onions')
        if 'black olives' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["pizzas"][SemanticFrame.pizza_num].info['toppings'].append('black olives')

        if 'root beer' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["drinks"].append('root beer')
        if 'orange soda' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["drinks"].append('orange soda')
        if 'lemon soda' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["drinks"].append('lemon soda')
        if 'mineral water' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["drinks"].append('mineral water')
        if 'ginger ale' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots["drinks"].append('ginger ale')
        if 'stuffed' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots['pizzas'][SemanticFrame.pizza_num].info["crust_type"] = 'stuffed'
        if 'deep dish' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots['pizzas'][SemanticFrame.pizza_num].info["crust_type"] = 'deep dish'
        if '4 cheese' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots['pizzas'][SemanticFrame.pizza_num].info["pizza_type"] = '4 cheese'
        if 'four cheese' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots['pizzas'][SemanticFrame.pizza_num].info["pizza_type"] = '4 cheese'
        if 'gluten free' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots['pizzas'][SemanticFrame.pizza_num].info["crust_type"] = 'gluten free'
        if 'veggie supreme' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots['pizzas'][SemanticFrame.pizza_num].info["pizza_type"] = 'veggie supreme'
        if 'meat lovers' in order:
          SemanticFrame.Intent = "INFORM"
          tokens_detected += 1
          SemanticFrame.Slots['pizzas'][SemanticFrame.pizza_num].info["pizza_type"] = 'meat lovers'

        for deop in DeliveryOptions:
          if deop in order:
            SemanticFrame.Intent = "INFORM"
            tokens_detected += 1
            SemanticFrame.Slots['delivery_option'] = deop

        #look for an address
        address_words = ['street', 'road','square', 'boulevard', 'highway', 'trail', 'court', 'parkway', 'drive', 'avenue', 'circle', 'st', 'rd', 'sq', 'blvd', 'hw', 'trl', 'ct', 'pkwy', 'dr', 'ave', 'cir']
        txt = r"\d{2,8}\D+(?:street|road|square|boulevard|highway|trail|court|parkway|drive|avenue|circle|st|rd|sq|blvd|hw|trl|ct|pkwy|dr|ave|cir)\W(?=\s|$)"
        address = re.findall(txt, order)
        # adding this to get words before address marker
        tokenized = order.split()
        # variable for full address
        full_address = ""
        for word in tokenized:
          #print('CURRENT WORD: ', word)
          if word in address_words:
            # add last word and stop
            full_address += word
            #print('CURRENT FULL ADDRESS: ', full_address)
            break
          else:
            # add on each word and a space
            word = word + ' '
            full_address += word
        #print('FULL ADDRESS: ', full_address)
        if address: #if list not empty
          #SemanticFrame.Slots["address"] = address[0]
          SemanticFrame.Slots["address"] = full_address
          tokens_detected += 1

        #look for a name
        txt = r"my name is [a-zA-Z]+"
        name = re.findall(txt, order)
        if name: #if list not empty
          n = name[0].split()
          SemanticFrame.Slots["name"] = n[3] #name will be fourth word
          tokens_detected +=1


        # go through each word and see if in any words recognized
        for token in tokens:
          if token in hello:
            SemanticFrame.Intent = "HELLO"
            tokens_detected += 1
          if token in goodbye:
            SemanticFrame.Intent = "GOODBYE"
            tokens_detected += 1
          if token in confirm:
            SemanticFrame.Intent = "CONFIRM"
            tokens_detected += 1
          if token in deny:
            SemanticFrame.Intent = "DENY"
            tokens_detected += 1
          """
          if token in pizza_types:
            SemanticFrame.Intent = "INFORM"
            tokens_detected += 1
            SemanticFrame.Slots["pizza_type"] = token
          if token in pizza_crusts:
            SemanticFrame.Intent = "INFORM"
            tokens_detected += 1
            SemanticFrame.Slots["crust_type"] = token
          if token in pizza_sizes:
            SemanticFrame.Intent = "INFORM"
            tokens_detected += 1
            SemanticFrame.Slots["pizza_size"] = token
          if token in delivery_options:
            SemanticFrame.Intent = "INFORM"
            tokens_detected += 1
            SemanticFrame.Slots["delivery_option"] = token
          """
          # check that phone number is 10 digits and then add it
          if token.isdigit() and len(token)==10:
            SemanticFrame.Intent = "INFORM"
            tokens_detected += 1
            SemanticFrame.Slots["phone_number"] = token
       
          #look at list of names to see if the user said a name
          if token in example_names:
            SemanticFrame.Intent = "INFORM"
            tokens_detected += 1
            SemanticFrame.Slots["name"] = token

          if token == 'cola':
            SemanticFrame.Intent = "INFORM"
            tokens_detected += 1
            SemanticFrame.Slots['drinks'].append('cola')

          for owt in one_word_toppings:
            if token == owt:
              SemanticFrame.Intent = "INFORM"
              tokens_detected += 1
              SemanticFrame.Slots['pizzas'][SemanticFrame.pizza_num].info['toppings'].append(owt)

        for wn in wrong_names:
          if wn in order:
            fix_wrong_names(wn, SemanticFrame, crf)
            tokens_detected = 1

  #remove duplicates from topping lists
  for pizz in SemanticFrame.Slots['pizzas']:
    pizz.info['toppings'] = list(set(pizz.info['toppings']))
    #remove partial names of toppings from topping list
    pizz.info['toppings'][:] = [toppi for toppi in pizz.info['toppings'] if toppi in menu_items]
    if pizz.info['crust_type'] not in menu_items:
      pizz.info['crust_type'] = ''
    if pizz.info['pizza_size'] not in menu_items:
      pizz.info['pizza_size'] = ''
    if pizz.info['pizza_type'] not in menu_items:
      pizz.info['pizza_type'] = ''

  #taking partial names of menu items out of Slots
  SemanticFrame.Slots['sides'][:] = [si for si in SemanticFrame.Slots['sides'] if si in menu_items]
  SemanticFrame.Slots['drinks'][:] = [dr for dr in SemanticFrame.Slots['drinks'] if dr in menu_items]

  if SemanticFrame.Slots['delivery_option'] not in DeliveryOptions:
    SemanticFrame.Slots['delivery_option'] = ''  

  # if nothing parses
  if tokens_detected == 0:
    chitchat(inputStr, SemanticFrame, crf)
    """
    inputStr = input('I\'m sorry, I didn\'t understand.' ) # Check your spelling and try again. ')
    # repeat function with new input
    nlu_parse(inputStr, SemanticFrame, crf)
    """
  return SemanticFrame

# used when user calls sthg by the wrong name (vege lovers, deepdish, etc.)
def fix_wrong_names(wrong_name, sframe, crf):
  right_name = wrong_names[wrong_name]
  st = "By " + wrong_name + " did you mean " + right_name + "? "
  answer = input(st + "\n")
  if answer in confirm:
    print("Okay, great! ")
    if right_name in pizza_types:
      sframe.Slots['pizzas'][sframe.pizza_num].info['pizza_type'] = right_name
    elif right_name in pizza_crusts:
      sframe.Slots['pizzas'][sframe.pizza_num].info['pizza_crust'] = right_name
  else:
    print("Okay. No problem. ")

def ground_and_price(SemanticFrame, crf):
  toppings, sides, drinks = 0, 0, 0

  #grounding
  # uppercase first letter of name
  username = SemanticFrame.Slots['name']
  first_letter = username[0].upper()
  username = username[1:]
  username = first_letter + username
  #print(name)
  s = "Okay " + username + ", your order is "
  pizza_counter = 0
  # increment through each pizza in the SF
  for pi in SemanticFrame.Slots['pizzas']:
    pizza_counter += 1
    # if more than 1 pizza in order:
    if pizza_counter == len(SemanticFrame.Slots['pizzas']) and len(SemanticFrame.Slots['pizzas']) != 1:
      s = s + "and "
    s = s + "a " + pi.info['pizza_size'] + " " + pi.info['crust_type'] + " " + pi.info['pizza_type'] + " pizza"
    if pi.info['toppings']: #if topping list not empty
      toppings = 1
      s = s + " with "
      for i in range(0, len(pi.info['toppings'])):
        if i == len(pi.info['toppings']):
          s = s + ' and ' + pi.info['toppings'][i]
        s = s + pi.info['toppings'][i] + ', '
      print('CURRENT STRING: ', s)
      #s = s.strip() #remove last space
      s = s[:len(s)] #remove last comma
      s = s + ", "
    else:
      s = s + ", "
    #s = s[:-2]
    #s = s + ', '
  s = s[:-2]
  s = s + ". "

  if 'sides' in SemanticFrame.Slots:
    if len(SemanticFrame.Slots['sides']) > 0:
      sides = 1
      s = s + "Your sides are "
      for item in SemanticFrame.Slots['sides']:
        s = s + item + ', '
      s = s.strip() #remove last space
      s = s[:len(s) - 1] #remove last comma
      s = s + ". "
  if 'drinks' in SemanticFrame.Slots:
    #print('DRINKS PRESENT!')
    if len(SemanticFrame.Slots['drinks']) > 0:
      drinks = 1
      s = s + "Your drinks are "
      for item in SemanticFrame.Slots['drinks']:
        s = s + item + ' '
      s = s.strip() #remove last space
      s = s[:len(s)] #remove last comma
      s = s + ". "
  if SemanticFrame.Slots['delivery_option'] == 'delivery':
    s = s + "The order is for delivery to " + str(SemanticFrame.Slots['address'] + ', ')
  else:
    s = s + "The order is for pickup, "

  #price
  price = 0.0
  for pi in SemanticFrame.Slots['pizzas']:
    price += PizzaCrusts[pi.info['pizza_size']][pi.info['crust_type']]
    for top in pi.info['toppings']:
      if top in Toppings:
        price += Toppings[top]

  price += DeliveryOptions[SemanticFrame.Slots['delivery_option']]

  if sides == 1:
    for si in SemanticFrame.Slots['sides']:
      if si in Sides:
        price = price + Sides[si]
  if drinks == 1:
    for d in SemanticFrame.Slots['drinks']:
      if d in Drinks:
        price = price + Drinks[d]

  formatted_price = "{:.2f}".format(price)
  s = s + "and will cost $" + str(formatted_price)+ ". Does that all sound correct?\n"
  answer = input(s)
  correct = 0
  for con in confirm:
    if con in answer:
      correct = 1
      print("Great! Thanks for ordering from Tony's! See you next time!")
  # figure out why this is just the same line over again, have to find a fix for this!
  if correct == 0:
    answer = input("Oh no! What do you want to change?\n")
    SemanticFrame = nlu_parse(answer, SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    nlg_question_order(SemanticFrame, crf)
    ground_and_price(SemanticFrame, crf)

#************************************************************************************************************************************************
# putting training here so don't have to redo every time:
  
#print("Training model...Will prompt when ready")
#folders = sys.argv[1:]
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#ensure the file is accessible
#!ls /content/gdrive/'My Drive'/'clams'/'Winter'/'LING575SDS'/PizzaBot
#current_dir = '/content/gdrive/My Drive/clams/Winter/LING575SDS/PizzaBot/training_data'
#current_dir = "C:\\Users\\smkea\\Desktop\\OG_pizza_bot\\data_train"
#crf_model = train_model(current_dir)

In [2]:
# separate block for training
current_dir = "C:\\Users\\smkea\\Desktop\\OG_pizza_bot\\data_train"
crf_model = train_model(current_dir)

HERES DIRNAME:  C:\Users\smkea\Desktop\OG_pizza_bot\data_train
HERES A FOLDER:  DATA1
HERES NEW DIR:  C:\Users\smkea\Desktop\OG_pizza_bot\data_train/DATA1
FILES:  ['group1.txt', 'group10.txt', 'group2.txt', 'group3.txt', 'group3updated.txt', 'group4.txt', 'group5.txt', 'group5updated.txt', 'group6.txt', 'group6updated.txt', 'group7.txt', 'group8.txt', 'group9.txt']
HERES A FOLDER:  DATA2
HERES NEW DIR:  C:\Users\smkea\Desktop\OG_pizza_bot\data_train/DATA2
FILES:  ['group1.txt', 'group10.txt', 'group2.txt', 'group3.txt', 'group3updated.txt', 'group4.txt', 'group5.txt', 'group5updated.txt', 'group6.txt', 'group6updated.txt', 'group7.txt', 'group8.txt', 'group9.txt']
HERES A FOLDER:  DATA3
HERES NEW DIR:  C:\Users\smkea\Desktop\OG_pizza_bot\data_train/DATA3
FILES:  ['group1.txt', 'group10.txt', 'group2.txt', 'group3.txt', 'group4.txt', 'group5.txt', 'group6.txt', 'group7.txt', 'group8.txt', 'group9.txt']


In [3]:
# -*- coding: utf-8 -*-
"""main.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1VjFYyzG_Ys0m7pREEHNxiP9P2UCUsmSo
"""

#main.py

import sys, re, os
get_ipython().system('pip install sklearn-crfsuite')
#import sklearn, sklearn_crfsuite


def main():

  print("Welcome to the Tony's Pizzeria Online Ordering System! You can enter 'repeat', 'start over', or 'cancel' at any time.")
  # declare semantic frame object
  semantic_frame = SemanticFrame()
  # declare a pizza object p
  p = Pizza()
  # add blank pizza object p to list of pizzas in semantic_frame.slots
  semantic_frame.Slots['pizzas'].append(p) 
  # set number of pizzas to zero
  semantic_frame.pizza_num = 0
  # start asking some questions
  nlg_question_order(semantic_frame, crf_model)
  # get the price totals and confirm everything?
  ground_and_price(semantic_frame, crf_model)

# TEST EDIT VIBESSSSSSSSSS

if __name__ == "__main__":
  main()

You should consider upgrading via the 'C:\Users\smkea\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Welcome to the Tony's Pizzeria Online Ordering System! You can enter 'repeat', 'start over', or 'cancel' at any time.
FULL ADDRESS:  small hawaiian thin crust for pickup please 
FULL ADDRESS:  thin crust 
FULL ADDRESS:  1234567890 
FULL ADDRESS:  my name is jeff 
DRINKS PRESENT!
Great! Thanks for ordering from Tony's! See you next time!
